In [0]:
###install pytorch in colab
# !pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
# !pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [1]:
##mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:
!ls '/content/drive/My Drive/DL_exp/sphere_shell_data_version3_using_uniform'

train_sp1_sh1.2_t1_16dim.csv  train_sp1_sh1.6_t1_16dim.csv
train_sp1_sh1.2_t1_2dim.csv   train_sp1_sh1.6_t1_2dim.csv
train_sp1_sh1.2_t1_32dim.csv  train_sp1_sh1.6_t1_32dim.csv
train_sp1_sh1.2_t1_4dim.csv   train_sp1_sh1.6_t1_4dim.csv
train_sp1_sh1.2_t1_64dim.csv  train_sp1_sh1.6_t1_64dim.csv
train_sp1_sh1.2_t1_8dim.csv   train_sp1_sh1.6_t1_8dim.csv
train_sp1_sh1.4_t1_16dim.csv  train_sp1_sh1.8_t1_16dim.csv
train_sp1_sh1.4_t1_2dim.csv   train_sp1_sh1.8_t1_2dim.csv
train_sp1_sh1.4_t1_32dim.csv  train_sp1_sh1.8_t1_32dim.csv
train_sp1_sh1.4_t1_4dim.csv   train_sp1_sh1.8_t1_4dim.csv
train_sp1_sh1.4_t1_64dim.csv  train_sp1_sh1.8_t1_64dim.csv
train_sp1_sh1.4_t1_8dim.csv   train_sp1_sh1.8_t1_8dim.csv
train_sp1_sh1.5_t1_16dim.csv  train_sp1_sh2_t1_16dim.csv
train_sp1_sh1.5_t1_2dim.csv   train_sp1_sh2_t1_2dim.csv
train_sp1_sh1.5_t1_32dim.csv  train_sp1_sh2_t1_32dim.csv
train_sp1_sh1.5_t1_4dim.csv   train_sp1_sh2_t1_4dim.csv
train_sp1_sh1.5_t1_64dim.csv  train_sp1_sh2_t1_64dim.csv
train_sp1_sh1.5

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
import json
import pandas as pd
import torch
import time
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn.init as init

In [0]:
base_path = '/content/drive/My Drive/DL_exp/'

In [0]:
### custom dataset class for dataset. read the csv in __init__ . 
# input : csv file path
# output: returns (data,label)
class load_dataset(Dataset):
    def __init__(self, data_path):
        self.samples = pd.read_csv(data_path).values

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx,:-1] ,self.samples[idx,-1]
      
### Multilayer Perceptron Model
# input : number of features (input), hidden nodes list , number of classes
class MultilayerPerceptron(torch.nn.Module):
  
    def __init__(self,num_features,hidden_nodes_list,num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        num_hidden_layes = len(hidden_nodes_list)
        self.hidden = torch.nn.ModuleList()
        
        self.hidden.append(torch.nn.Linear(num_features, hidden_nodes_list[0]))
        for k in range(num_hidden_layes-1):
            self.hidden.append(torch.nn.Linear(hidden_nodes_list[k], hidden_nodes_list[k+1]))    
        self.hidden.append(torch.nn.Linear(hidden_nodes_list[num_hidden_layes-1], num_classes))
        
    # input : features
    # output: logits , probabilities
    def forward(self, x):
        out = x
        for layer in self.hidden[:-1]:
          out = layer(out)
          out = F.relu(out)

        logits = self.hidden[-1](out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas
      
### weight initialization function
# use : model.apply(init_weights)
def init_weights(m):
  if isinstance(m, torch.nn.Linear):
    if initialisation_method=='xavier':
      init.xavier_uniform_(m.weight)
    if initialisation_method=='he':
      init.kaiming_uniform_(m.weight)
    m.bias.data.fill_(0.01)
    
### function to compute the accuracy
# input : model, data of type DataLoader
# output: cost (log loss), accuracy 
def compute_accuracy(net, data_loader):
    net.eval()
    cost, correct_pred, num_examples = 0, 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.float().to(device)
            targets = targets.long().to(device)
            logits, probas = net(features)
            cost += F.cross_entropy(logits, targets) * targets.size(0)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return cost/num_examples , correct_pred.float()/num_examples * 100

In [0]:
### Training of multiple models

# Input: csv file path contain sphere-shell d dimension features,label data. 
# Its name is like 'train_sp1_sh1.8_t1_2dim.csv' where sphere, shell radius, shell thickness and dimension are variable that have to specify.
radius_of_sphere = 1
radius_of_shell = 1.8
thickness_of_shell = 1
dimentions = [2,4,8,16,32,64]          # list of dimention of the data
# Output: upade 'uniform_result.json' file with the data, trained model details, accuracy, epoch and loss
# also update 'loss_uniform.json' file which have list of [{epoch},{loss}] data of trained model 
result_folder = 'result_Hypothesis_B9/'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
start_time = time.time()

with open(base_path+result_folder+'loss_uniform.json', mode='a+') as readjson:
  try: 
    loss = json.loads(readjson.read())
  except ValueError: 
    loss = {}
          
for dim in dimentions:
  csv_path = base_path+'sphere_shell_data_version3_using_uniform/train_sp'+ str(radius_of_sphere) + '_sh'+ str(radius_of_shell) + '_t' + str(thickness_of_shell)+ '_' + str(dim) + 'dim.csv'
  dataset = load_dataset(csv_path)
  size = len(dataset)
  train_dataloader = DataLoader(dataset, batch_size=size, shuffle=True)


  tmp_h = [2,4,6,8,12,16,24,32,48,64,128,256]             #list of total number of nodes in hidden layers
  tmp_i = [100,200,300,400,500,600,700,800,900,1000]      #list of random seed for weight initialization

  for t_h in tmp_h:
    for t_i in tmp_i:

      #architecture
      num_features  = next(iter(dataset))[0].shape[0]        # Input data dimention
      hidden_nodes_list   = [t_h]                            # List of number of nodes at each hidden layer
      num_classes   = 2                                      # The number of output classes. In this case, 0 and 1
      
      # 'xavier' : Xavier Initialisation
      # 'he' : He Initialisation
      initialisation_method = 'xavier'
      
      # 'sgd' : SGD (lr) 
      # 'sgdwm' : SGD with Momentum (lr, momentum)
      # 'adagrad' : AdaGrad
      # 'adam' : Adam
      # 'ngd' : Natural gradient descent
      # 'l1' : L1 Regularisation
      # 'l2' : L2 Regularisation
      # 'pathnorm' : PathNorm Regularisation
      # 'spectralnorm' : Spectral norm Regularisation
      optimisation_method = 'sgdwm'  

      # Hyperparameters
      random_seed = t_i
      learning_rate = 0.05
            
      torch.manual_seed(random_seed)
      model = MultilayerPerceptron(num_features,hidden_nodes_list,num_classes)
      model.apply(init_weights)
      model = model.to(device)

      if optimisation_method=='sgd' or optimisation_method=='ngd':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
      if optimisation_method=='sgdwm':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
      if optimisation_method=='adagrad':
        optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
      if optimisation_method=='adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08)

      model_loss = []
      epoch = 0

      with torch.set_grad_enabled(False):
        cost,best_acc = compute_accuracy(model, train_dataloader)
      model_loss.append([epoch,cost.data.tolist()])
      
      count = 1
      prev_acc=best_acc
      best_epoch = epoch
      best_cost = cost

      while True:
          model.train()
          for batch_idx, (features, targets) in enumerate(train_dataloader):

              features = features.float().to(device)
              targets = targets.long().to(device)

              ### FORWARD AND BACK PROP
              logits, probas = model(features)
              cost = F.cross_entropy(logits, targets)
              optimizer.zero_grad()

              cost.backward()

              ### UPDATE MODEL PARAMETERS
              optimizer.step()


          with torch.set_grad_enabled(False):
              cost,acc = compute_accuracy(model, train_dataloader)
              
          epoch+=1
          model_loss.append([epoch,cost.data.tolist()])
          
          # Stopping conditions
          if prev_acc==acc:
            count+=1
          else:
            prev_acc=acc
            count=1
          if (epoch>50 and best_acc-acc>=5) or count==20 or epoch==400:
            break
          if acc>best_acc:
            best_acc=acc
            best_epoch = epoch
            best_cost = cost
 
      print('Epoch: %03d | Accuracy: %.2f%% | Cost: %.4f' % (best_epoch,best_acc,best_cost))
  
      if len(hidden_nodes_list)==1:
        loss.update({'sh'+ str(radius_of_shell)+'dim'+str(dim)+'h'+str(t_h)+'i'+str(t_i):model_loss})
      else:
        loss.update({'sh'+ str(radius_of_shell)+'dim'+str(dim)+'h'+str(t_h)+'i'+str(t_i)+'l2':model_loss})
    
      with open(base_path+result_folder+'uniform_result.json', mode='a+') as readjson:
        try: 
          result = json.loads(readjson.read())
        except ValueError: 
          result = []  
       
      with open(base_path+result_folder+'uniform_result.json', mode='w') as feedjson:
        entry = {'sphere_radius': 1, 'shell_radius': radius_of_shell, 'thickness' : 1, 'dimension':dim, 'n_nodes':hidden_nodes_list, 'random_seed':t_i, 'epoch':best_epoch, 'accuracy':best_acc.data.tolist(), 'cost':best_cost.data.tolist()}
        result.append(entry)
        json.dump(result, feedjson)

with open(base_path+result_folder+'loss_uniform.json', mode='w') as feedjson:
  json.dump(loss, feedjson)

# print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))